In [1]:
import uproot

# filename = 'silene/1-2-8_rel/results-master.root'
filename = 'silene/calc 3/results-silene-220323.root'

file = uproot.open(filename)
ttree = file['Universal Ntuple']

#                             mm      mm      mm      mm      mm      mm      cm    MeV
data = ttree.arrays(["type", "xPos", "yPos", "zPos", "xDir", "yDir", "zDir", "len", "E", "histN", "prevProcID", "postProcID"], library='pd')

In [2]:
# Какие типы бывают?
data['type'].unique()

array([31, 21, 11, 10, 20, 30], dtype=int32)

Для источника silene имеем 6 типов частиц.
10 - нейтроны     в SileneOuterCadRingPV
11 - гамма-кванты в SileneOuterCadRingPV
20 - нейтроны     в DetPV
21 - гамма-кванты в DetPV
30 - нейтроны     в BotDetPV
31 - гамма-кванты в BotDetPV

Отфильтруем частицы, которые сможем использовать для расчета утечки. 

Сначала разберемся с утечкой нейтронов с поверхности кадмиевого экрана, то есть с типом 10.

In [3]:
data_n_cad = data[data['type'] == 10]
# 106354 нейтрона из 3 млн историй
# data_n_cad = data_n_cad[data_n_cad["postProcID"] == 1]
res = data_n_cad[data_n_cad['histN'] > 10000]

res

,type,xPos,yPos,zPos,xDir,yDir,zDir,len,E,histN,prevProcID,postProcID
3919,10,-59.513359,-426.164581,1408.778358,0.340191,-0.888351,0.308389,0.084037,3.286727e+00,14689,1,1
3948,10,192.614675,384.782636,712.815062,-0.036588,0.776798,-0.628686,0.020624,3.650216e-07,14695,1,6
6491,10,394.806275,-171.131808,1079.446990,0.839040,-0.185615,-0.511429,0.016742,2.755628e-07,10418,1,6
6720,10,225.670776,366.375205,925.467334,0.410018,0.849945,-0.330876,0.074570,4.849234e+00,10506,1,1
6814,10,-74.447782,423.810828,1131.456239,0.567700,0.779585,-0.264506,0.104448,1.687048e+00,10558,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6530018,10,-298.777265,-309.661486,962.956729,-0.352807,-0.587187,0.728518,0.002282,3.204855e-08,2999903,1,6
6530047,10,427.887175,45.504457,1566.658658,0.975999,0.217407,0.012620,0.070456,1.095306e-01,2999904,1,1
6530049,10,-408.186300,136.169139,1201.141161,-0.930745,0.248576,-0.268187,0.000056,5.681284e-08,2999905,1,6
6530140,10,50.461360,427.330950,1568.734275,0.361882,0.384775,0.849111,0.001252,1.253660e-07,2999944,1,6


In [92]:
import math

def mult_vec(vec1, vec2):
    return vec1[0] * vec2[0] + vec1[1] * vec2[1] + vec1[2] * vec2[2]

def norm_direction(arg):
    # type	xPos	yPos	zPos	xDir	yDir	zDir	len	  E
    #    0     1       2       3       4       5       6      7   8
    # Для цилиндра с образующей вдоль OZ внешняя нормаль это всегда будет (xPos, yPos, 0)
    # print(mult_vec([arg[1], arg[2], 0], [arg[4], arg[5], arg[6]]))
    return True if mult_vec([arg[1], arg[2], 0], arg[4:7]) > 0 else False

In [96]:
out_filter = data_n_cad.apply(norm_direction, axis=1)

In [99]:
dnc_out = data_n_cad[out_filter == True]
dnc_in = data_n_cad[out_filter == False]

In [104]:
# Кадмиевый экран - цилиндрический слой с центром в координатах (0, 0, 1162) мм.
# Параметры rmin="43.03" rmax="43.1" startphi="0.0" deltaphi="360.0" aunit="deg" z="100.0" lunit="cm"

# Проверим, что у всех радиус менее 43.1 и более 43.1
rad = dnc_out.apply(
    lambda x: math.sqrt(x[1]**2 + x[2]**2),
    axis=1
)

In [107]:
rad[rad == 430.3]

10         430.3
228        430.3
236        430.3
642        430.3
644        430.3
           ...  
6526976    430.3
6527175    430.3
6527295    430.3
6527328    430.3
6527531    430.3
Length: 72749, dtype: float64

In [46]:
def get_cos ( text ):     # Function name helps explain the code.
    if text[1] == ' ':
        return int( text[3:7] )  # Clearer and shorter than append().
    else:
        return int( text[4:8] )  # Clearer and shorter than append().
    
# df['first_year'] = df['years'].apply( get_first_year )

# df.iloc[[0,14441],:] # same check as before